In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter
from sklearn.model_selection import train_test_split
import nltk
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf


NUM_WORKERS = 8
TEST_SIZE = 0.3
LABEL_COLUMN_NAME = "oh_label"
TEXT_COLUMN_NAME = "Text"
DATASET_PATH = "../Data/ver1.csv"

df= pd.read_csv(DATASET_PATH, index_col=False)
df = df.dropna()
df.drop(["Unnamed: 0", "index"],axis=1, inplace = True)

In [3]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [2]:
doc2vec_model = Doc2Vec([TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)], vector_size=300, window=50, min_count=5, workers=NUM_WORKERS)
stop_words = set(nltk.corpus.stopwords.words("english"))
X = df[TEXT_COLUMN_NAME].swifter.apply(lambda text : doc2vec_model.infer_vector([word for word in nltk.word_tokenize(text) if word not in stop_words]))
y = df[LABEL_COLUMN_NAME]

RuntimeError: you must first build vocabulary before training the model

In [ ]:
x_neg_weight = y[y==1].shape
x_pos_weight = y[y==0].shape
x_neg_weight[0]/x_pos_weight[0]
weights = {0: (x_neg_weight[0]/(x_pos_weight[0]+x_neg_weight[0])),1: (x_pos_weight[0]/(x_pos_weight[0]+x_neg_weight[0]))}
print(weights)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=1)
x_train = np.expand_dims(np.stack(x_train),axis=2)
x_test = np.expand_dims(np.stack(x_test),axis=2)

In [ ]:
x_train.squeeze().shape

In [ ]:
# strategy = tf.distribute.MirroredStrategy()
model = Sequential()
model.add(Dense(200, activation="relu",input_shape=(300,1)))
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
history = model.fit(x_train,y_train,batch_size=200,epochs=500,
                    validation_split=0.05,class_weight=weights.to_dict(),
                    use_multiprocessing=True,workers=8,callbacks=[callback])